# Animation of a mesh attribute

In [ ]:
import k3d
import vtk
import glob
import os
from vtk.util import numpy_support
import numpy as np

In [ ]:
reader = vtk.vtkSTLReader()
reader.SetFileName('./data/c0006.stl')
reader.Update()
geometry_wss = reader.GetOutput()

In [ ]:
wss = np.load('./data/wss.npy',allow_pickle=True).item()

In [ ]:
wss_reduced = {}

for k in wss.keys():
    if k % 50 == 0:
        wss_reduced[str(k * 0.01)] = wss[k]
        
print(len(wss_reduced.keys()))

In [ ]:
vertices = numpy_support.vtk_to_numpy(geometry_wss.GetPoints().GetData())
indices = numpy_support.vtk_to_numpy(geometry_wss.GetPolys().GetData()).astype(np.uint32).reshape(-1, 4)[:, 1:4]

plot = k3d.plot()
plot += k3d.mesh(vertices, indices, 
                 color_map=k3d.colormaps.paraview_color_maps.Jet, 
                 attribute=wss_reduced, 
                 color_range=[0,3], 
                 )
plot.display()

In [ ]:
plot.camera_auto_fit = False
plot.grid_auto_fit = False

In [ ]:
from time import sleep

for t in range(1120):
    plot.time = t / 100.0
    sleep(1.0/30.0)